![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+ML%2FSolution+Prototypes%2Fdocument-processing&file=6-document-comparison.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/6-document-comparison.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520ML%2FSolution%2520Prototypes%2Fdocument-processing%2F6-document-comparison.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20ML/Solution%20Prototypes/document-processing/6-document-comparison.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20ML/Solution%20Prototypes/document-processing/6-document-comparison.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Comparing Documents For Anomalies

> This workflow is part of a series of workflows for the solution prototype: [Document Processing With Generative AI: Parse, Extract, Validate Authenticity, and More](./readme.md)

After documents are flagged as anomalous, the next step is to pinpoint the specific layout or formatting elements that suggest potential fraud. A multimodal generative model, such as Gemini on Vertex AI, is used to perform an initial evaluation and highlight potential indicators of fraud. This process is then integrated directly into BigQuery, further enriching the data.

This integration is enabled by using the Gemini API on Vertex AI to generate comparisons with [multimodal prompts](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#sample-requests-text-gen-multimodal-prompt) via the [Google Gen AI SDK](https://cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview). The [generating](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference) of comparisons is scaled by using the Gemini API within BigQuery with functions like [ML.GENERATE_TEXT](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text).

## Setup

Note that this notebook expects to use a local virtual environment with the `./requirements.txt` installed.  

A potential workaround if using this notebook standalone is running:

>```python
>pip install -r requirements.txt
>```

And then restart the kernel.

In [6]:
# package imports for this work
import os, subprocess

from IPython.display import display, Image, Markdown
import ipywidgets
import fitz # PyMuPDF

from google.cloud import storage
from google.cloud import bigquery
from google import genai

In [2]:
# what project are we working in?
PROJECT_ID = subprocess.run(['gcloud', 'config', 'get-value', 'project'], capture_output=True, text=True, check=True).stdout.strip()
PROJECT_ID

'statmike-mlops-349915'

In [3]:
LOCATION = 'us-central1'

SERIES = 'applied-ml-solution-prototypes'
EXPERIMENT = 'document-processing'
GCS_BUCKET = PROJECT_ID # bucket has same name as project here

In [4]:
# setup google cloud storage client
gcs = storage.Client(project = PROJECT_ID)
bucket = gcs.bucket(GCS_BUCKET)

# setup genai client
genai_client = genai.Client(vertexai = True, project = PROJECT_ID, location = LOCATION)

# setup google cloud bigquery client
bq = bigquery.Client(project = PROJECT_ID)

# load the bigquery magics for jupyter with:
%load_ext bigquery_magics

---
## Visually Compare Documents

Load an view the document from the previous workflow and compare it to the template for the same vendor.

### Read And Process Documents

Read the documents from GCS in the PDF format and prepare locally as bytes objects for the PDF and a PNG of the first page.

In [7]:
query_doc = 'vendor_5_invoice_2.pdf'
query_vendor = 'vendor_5'

bucket = gcs.bucket(GCS_BUCKET)
blob_prefix = f"{SERIES}/{EXPERIMENT}/"

In [9]:
query_file= bucket.blob(blob_prefix + f'{query_vendor}/fake_invoices/{query_doc}')
compare_file = bucket.blob(blob_prefix + f'{query_vendor}/template/template.pdf')

In [10]:
query_file_bytes = query_file.download_as_bytes()
compare_file_bytes = compare_file.download_as_bytes()

In [11]:
query_pdf = fitz.open(stream = query_file_bytes, filetype = 'pdf')
compare_pdf = fitz.open(stream = compare_file_bytes, filetype = 'pdf')

In [14]:
query_image_bytes = query_pdf.load_page(0).get_pixmap(dpi = 150).tobytes(output = 'png')
compare_image_bytes = compare_pdf.load_page(0).get_pixmap(dpi = 150).tobytes(output = 'png')

### View Documents Side-By-Side

In [16]:
display(
    ipywidgets.HBox(
        [
            ipywidgets.VBox(
                [
                    ipywidgets.HTML(value = f"<div style='text-align: center; margin-bottom: 5px;'><b>Query Document</b></div>"),
                    ipywidgets.Image(value = query_image_bytes, format = 'png', width = 400) 
                ],
                layout = {'align_items': 'center'}
            ),
            ipywidgets.VBox(
                [
                    ipywidgets.HTML(value = f"<div style='text-align: center; margin-bottom: 5px;'><b>Compare Document</b></div>"),
                    ipywidgets.Image(value = compare_image_bytes, format = 'png', width = 400) 
                ],
                layout = {'align_items': 'center'}
            )
        ],
        layout = {'justify_content': 'space-around'}
    )
)

### Use Gemini With Vertex AI To Compare Documents

In [38]:
prompt_query_image = genai.types.Part.from_bytes(
    data = query_image_bytes,
    mime_type = 'image/png'
)
prompt_compare_image = genai.types.Part.from_bytes(
    data = compare_image_bytes,
    mime_type = 'image/png'
)

In [39]:
response = genai_client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = [
        prompt_query_image,
        prompt_compare_image,
        f"""
        These two images represent two documents from the same vendor but it seems that some parts of the formating and/or layout have changed.
        The first image is the new version of the document.
        The second image is a known authentic document from the same vendor.
        Describe the differences in appearance for the layout and formatting.
        """
    ],
    config = genai.types.GenerateContentConfig(
        system_instruction = f"""
        Your job is to compare document for subtle clues that indicate new documents might be fraudulent.
        These are invoices and the id, date, bill to adddress, lines items are expected to change so ignore those.
        The formatting and layout are the primary things to evaluate for differences.
        Look for differences in all visual aspects from placement, fonts, colors, spacing, etc.
        """
    )
)

In [40]:
Markdown(response.text)

Okay, here's a breakdown of the differences I've identified between the two invoice images, focusing on formatting and layout:

**General Layout and Spacing**

*   **Logo:** The logo in the first image is noticeably larger than in the second image. The image also sits closer to the top of the page.
*   **Bill To Address:** The bill to address moves from Seattle to Miami, which is expected, but there are other format changes too. There is more spacing between the Bill To and the address lines. Also, the Bill To title has changed fonts to a thicker one.
*   **Tax Calculation:** The first image has 8% tax while the second has 6%, which has an effect on the subtotal.

**Font and Text Styles**

*   **Bill To:** The bill to has changed to bold font in the first image.

**Table Differences**

*   **Third Row Quantity:** The quantity is different in the third row. In the first image it is 40 while the second is 60.

#### This Also Works With PDF Versions

In [41]:
prompt_query_image = genai.types.Part.from_bytes(
    data = query_file_bytes,
    mime_type = 'application/pdf'
)
prompt_compare_image = genai.types.Part.from_bytes(
    data = compare_file_bytes,
    mime_type = 'application/pdf'
)

In [42]:
response = genai_client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = [
        prompt_query_image,
        prompt_compare_image,
        f"""
        These two images represent two documents from the same vendor but it seems that some parts of the formating and/or layout have changed.
        The first image is the new version of the document.
        The second image is a known authentic document from the same vendor.
        Describe the differences in appearance for the layout and formatting.
        """
    ],
    config = genai.types.GenerateContentConfig(
        system_instruction = f"""
        Your job is to compare document for subtle clues that indicate new documents might be fraudulent.
        These are invoices and the id, date, bill to adddress, lines items are expected to change so ignore those.
        The formatting and layout are the primary things to evaluate for differences.
        Look for differences in all visual aspects from placement, fonts, colors, spacing, etc.
        """
    )
)

In [43]:
Markdown(response.text)

Okay, I've compared the two invoices and here's a breakdown of the visual differences I've identified that might indicate a fraudulent document:

**General Layout and Spacing:**

*   **Invoice Header Placement:** In the original invoice, the "Invoice" title is closer to the right edge of the document. In the new document, the title is closer to the company logo.

*   **Spacing around "Cyberdyne Systems" Logo:** The new document has less space between the logo and the "Cyberdyne Systems" text below it compared to the original.

*   **Spacing Bill To Address:** The spacing between the "Bill to" header and the company logo, and between the address and the product lines, looks slightly different. It seems that the new document has reduced the spacing.

**Font and Text Styling:**

*   **"Cyberdyne Systems" Font/Weight:** There might be a slight difference in the font or weight used for "Cyberdyne Systems" below the logo. The original document seems bolder.

*   **Font Size/Weight in Address Section:** The font size in the address section of the new document seems smaller than the original one. The original has a slightly heavier font.

*   **Font Size/Weight in description columns:** The font size in the description columns seems smaller than the original one. The original has a slightly heavier font.

**Table and Line Items:**

*   **Line Spacing in Description:** The line spacing within the "Description" column might be slightly tighter in the original document.

**Other Notable Differences:**

*   **Tax rate:** The tax rate has changed from 8% to 6%.
*   **Quantity column:** The quantity for the last item has changed from 40 to 60.

**Summary of Potential Red Flags:**

*   Subtle changes in spacing and font styling.
*   Changes in tax rate and quantity column.


#### This Also Work With Files Directly From GCS

In [44]:
prompt_query = genai.types.Part.from_uri(
    file_uri = f"gs://{bucket.name}/{query_file.name}",
    mime_type = 'application/pdf'
)
prompt_compare = genai.types.Part.from_uri(
    file_uri = f"gs://{bucket.name}/{compare_file.name}",
    mime_type = 'application/pdf'
)

In [45]:
response = genai_client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = [
        prompt_query,
        prompt_compare,
        f"""
        These two images represent two documents from the same vendor but it seems that some parts of the formating and/or layout have changed.
        The first image is the new version of the document.
        The second image is a known authentic document from the same vendor.
        Describe the differences in appearance for the layout and formatting.
        """
    ],
    config = genai.types.GenerateContentConfig(
        system_instruction = f"""
        Your job is to compare document for subtle clues that indicate new documents might be fraudulent.
        These are invoices and the id, date, bill to adddress, lines items are expected to change so ignore those.
        The formatting and layout are the primary things to evaluate for differences.
        Look for differences in all visual aspects from placement, fonts, colors, spacing, etc.
        """
    )
)

In [ ]:
Markdown(response.text)

Okay, I've analyzed the two invoice images and here's a breakdown of the differences I've identified:

**General Layout and Spacing:**

*   **Overall:** The general layout is the same, but the spacing seems to be different, the old document is more compact.

**Font and Text:**

*   **Font Size/Weight:** There seem to be some slight differences in the font size or weight used for certain sections of the document. It's subtle, but the older document looks like it's using a slightly heavier font in places (like "Cyberdyne Systems").
*   **Text Placement:** The location of elements like the invoice number, date, and due date also appear to be slightly different. In the old document is spaced farther away from the title

**Other Visual Elements:**

*   **Logo:** The logo "Cyberdyne Systems" looks like it might be a slightly different shade of blue or a bit bolder in the older document.

**Details that Suggest Potential Issues:**

*   **Tax Rate:** The tax rate is different between the two invoices (8% vs 6%). While tax rates can change, it's something to verify.
*   **Bill To Address:** The "Bill to" address is different. This is normal if the invoices are for different customers, but it should be verified against customer records.

**In summary:** The differences are subtle, primarily involving spacing, and minor variations in font appearance. The changes to the bill to address and tax require closer examination and verification against external information.


---
---

# NOTE: CONTENT BELOW IN DEVELOPMENT

The content below is intended to show using Gemini Models withing BigQuery with ML.GENERATE_TEXT doing the comparison task above.  At this time it appears to be limited to a single document per prompt so the content is paused as a workaround is figured out.  A definite workaround is using batch processing the Vertex AI where the input and output tables can be BigQuery based.  

---
---

## Use Gemini Directly Within BigQuery

The same Gemini model on Vertex AI can used directly within BigQuery.  Just as we used the Document AI custom extractor and Vertex AI embeddings API on earlier workflows in the project we can also use Gemini models.

**BigQuery Cloud Resource Connection**

The earlier workflows setup a resource connection and already assigned the role `roles/aiplatform.user` to the associated service account.  This is the same requirment for using Gemini models on Vertex AI with BigQuery.

### Create A BigQuery Remote Model For The Gemini Model

We need to create a connection to the Verex AI Gemini Model endpoint via the [CREATE MODEL](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model) statement - [example for Gemini Flash without tuning](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model#create_a_model_without_tuning).

In [48]:
%%bigquery
CREATE OR REPLACE MODEL `statmike-mlops-349915.solution_prototype_document_processing.gemini-2_0-flash`
    REMOTE WITH CONNECTION `statmike-mlops-349915.us.document-processing`
    OPTIONS(
        ENDPOINT = 'gemini-2.0-flash'
    )

Query is running:   0%|          |

""


### Generate Comparison With BigQuery

Use the [ML.GENERATE_TEXT](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-text#examples) function with the model to generate comparisons.

In [50]:
%%bigquery
SELECT *
FROM `statmike-mlops-349915.solution_prototype_document_processing.unknown_authenticity`
LIMIT 1

Query is running:   0%|          |

Downloading:   0%|          |

,ml_process_document_result,ml_process_document_status,vendor_name,vendor_address,company_name,company_address,invoice_id,invoice_total,line_item,uri,updated,vendor,embedding,anomaly_distance,anomaly_z_score,anomaly_decision
0,"{""mimeType"":""application/pdf""}",,None,None,None,None,None,None,[],gs://statmike-mlops-349915/applied-ml-solution...,2025-04-23 20:55:09.934000+00:00,vendor_8,"[0.0160381682, 0.0116377119, 0.00194062886, 0....",-0.460991,55.873966,True


In [60]:
%%bigquery
SELECT *
FROM ML.GENERATE_TEXT(
    MODEL `statmike-mlops-349915.solution_prototype_document_processing.gemini-2_0-flash`,
    (
        SELECT uri, content_type
        FROM `statmike-mlops-349915.solution_prototype_document_processing.source_documents`
        LIMIT 2
    ),
    STRUCT (
        'Compare the layout and format of these documents and identify differences' AS PROMPT,
        TRUE AS FLATTEN_JSON_OUTPUT
    )
)



Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,uri,content_type
0,There are no differences between the documents...,None,,gs://statmike-mlops-349915/applied-ml-solution...,application/pdf
1,The provided images consist of a single docume...,None,,gs://statmike-mlops-349915/applied-ml-solution...,application/pdf


In [61]:
%%bigquery
SELECT *
FROM ML.GENERATE_TEXT(
    MODEL `statmike-mlops-349915.solution_prototype_document_processing.gemini-2_0-flash`,
    (
        SELECT sd.uri, sd.content_type
        FROM (
            SELECT *
            FROM `statmike-mlops-349915.solution_prototype_document_processing.unknown_authenticity`
            WHERE uri LIKE '%vendor_5_invoice_2%'
        ) ua
        JOIN `statmike-mlops-349915.solution_prototype_document_processing.source_documents` sd
        ON
            ua.uri = sd.uri
            OR sd.uri = REPLACE(ua.uri, '/fake_invoices/', '/invoices/')
    ),
    STRUCT (
        'Compare the layout and format of these documents and identify differences' AS PROMPT,
        TRUE AS FLATTEN_JSON_OUTPUT
    )
)

Executing query with job ID: c0dd6241-c260-419a-a6dc-4b4f3edca77d
Query executing: 0.62s


ERROR:
 400 column prompt is not found in the input of ML.GENERATE_TEXT; reason: invalidQuery, location: query, message: column prompt is not found in the input of ML.GENERATE_TEXT

Location: US
Job ID: c0dd6241-c260-419a-a6dc-4b4f3edca77d

